In [ ]:
import os
import torch
from transformers import Trainer, TrainingArguments, T5ForConditionalGeneration, T5Tokenizer, DataCollatorForSeq2Seq
from transformers import EarlyStoppingCallback
import pandas as pd
from sklearn.model_selection import train_test_split


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/medquad.csv")
df.head()


,question,answer,source,focus_area
0,What is (are) Glaucoma ?,Glaucoma is a group of diseases that can damag...,NIHSeniorHealth,Glaucoma
1,What causes Glaucoma ?,"Nearly 2.7 million people have glaucoma, a lea...",NIHSeniorHealth,Glaucoma
2,What are the symptoms of Glaucoma ?,Symptoms of Glaucoma Glaucoma can develop in ...,NIHSeniorHealth,Glaucoma
3,What are the treatments for Glaucoma ?,"Although open-angle glaucoma cannot be cured, ...",NIHSeniorHealth,Glaucoma
4,What is (are) Glaucoma ?,Glaucoma is a group of diseases that can damag...,NIHSeniorHealth,Glaucoma


In [ ]:
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16393 entries, 0 to 16411
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   question    16393 non-null  object
 1   answer      16393 non-null  object
 2   source      16393 non-null  object
 3   focus_area  16393 non-null  object
dtypes: object(4)
memory usage: 640.4+ KB


In [ ]:
import re
def preprocess_text(text):
    text = text.lower().strip()  # Lowercase and remove whitespace
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)  # Remove special characters
    return text

# Apply preprocessing
df['question'] = df['question'].apply(preprocess_text)
df['answer'] = df['answer'].apply(preprocess_text)

In [ ]:
# Split into training and validation sets
train_data, val_data = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
# Initialize T5 model and tokenizer
model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')


In [ ]:
# Function for tokenization
def preprocess_function(examples):
    inputs = ["question: " + q + " provide a detailed answer" for q in examples['question']]
    targets = [a for a in examples['answer']]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding='max_length', return_tensors="pt")
    labels = tokenizer(targets, max_length=512, truncation=True, padding='max_length', return_tensors="pt")['input_ids']
    model_inputs['labels'] = labels
    return model_inputs

# Preprocess train and validation data
train_encodings = preprocess_function(train_data)
val_encodings = preprocess_function(val_data)


In [ ]:
# Directory setup for saving results
results_dir = "/content/drive/MyDrive/colab_results"
logs_dir = "/content/drive/MyDrive/colab_logs"
os.makedirs(results_dir, exist_ok=True)
os.makedirs(logs_dir, exist_ok=True)

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir=results_dir,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-4,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=2,
    num_train_epochs=20,
    weight_decay=0.01,
    save_total_limit=3,
    logging_dir=logs_dir,
    logging_steps=100,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False
)
# Custom Dataset class for MedQuad
class MedQuadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings['input_ids'])

# Create PyTorch datasets for training and validation
train_dataset = MedQuadDataset(train_encodings)
val_dataset = MedQuadDataset(val_encodings)

# Define the data collator for T5
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# Initialize Trainer with early stopping callback
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)
# Path to the last checkpoint from the first epoch
latest_checkpoint = "/content/drive/MyDrive/checkpoint-6970/checkpoint-6970"

# Fine-tune the model
trainer.train(resume_from_checkpoint=latest_checkpoint)

# Save the fine-tuned model
model.save_pretrained('/content/drive/MyDrive/t5_finetuned_medquad')
tokenizer.save_pretrained('/content/drive/MyDrive/t5_finetuned_medquad')

print("Model fine-tuning completed and saved.")



/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].
/usr/local/lib/python3.10/dist-packages/transformers/trainer.py:3098: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untru

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/usr/local/lib/python3.10/dist-packages/transformers/trainer.py:2833: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint_rng_state = torch.load(rng_file)
<ipython-inpu

Epoch,Training Loss,Validation Loss
18,0.795300,0.821854
19,0.799100,0.821469
20,0.793100,0.821253


<ipython-input-21-39a843b66663>:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-21-39a843b66663>:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-21-39a843b66663>:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
There were missing keys in the checkpoint model loaded: ['encoder.embe

Model fine-tuning completed and saved.


In [ ]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load the fine-tuned model and tokenizer
model = T5ForConditionalGeneration.from_pretrained('/content/drive/MyDrive/t5_finetuned_medquad')
tokenizer = T5Tokenizer.from_pretrained('/content/drive/MyDrive/t5_finetuned_medquad')

# Set the model to evaluation mode and load it onto the available device (GPU if possible)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()

# Function to generate an answer
def generate_answer(question, model, tokenizer, max_length=100):
    # Prepare the question input in the T5 format
    input_text = "question: " + question
    inputs = tokenizer(input_text, return_tensors="pt").to(device)

    # Generate the answer
    outputs = model.generate(
        inputs['input_ids'],
        max_length=max_length,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        top_k=50,
        top_p=0.95,
        temperature=0.7,
        do_sample=True,
    )

    # Decode the generated answer
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

# Test the model with questions from the dataset
sample_questions = [
    "What is glaucoma?",
    "What are the symptoms of diabetes?",
    "Who is at risk of prostate cancer?",
    "What is anxiety disorder?"
]

for question in sample_questions:
    answer = generate_answer(question, model, tokenizer)
    print("Question:", question)
    print("Answer:", answer)
    print("-" * 30)


Question: What is glaucoma?
Answer: glaucoma is a disorder in which large amounts of light are released from the bloodstream and are not damaged by toxins that affect the retina people who have ocular neoplasm may have low vision because they are able to see the light through the lens at varying speeds the resulting amount of shingles is usually the result of an abnormally large amount to the eye in dilated eyelids or ct scans that can
------------------------------
Question: What are the symptoms of diabetes?
Answer: type 1 diabetes is a disorder in which if you have diabetes you may have trouble gaining weight and having too much weight with aging and achieving the goals of reducing your blood glucose or having hyperglycemia
------------------------------
Question: Who is at risk of prostate cancer?
Answer: prostate cancer is a rare disease that can affect the risk of prostate disease prostate treatment may increase the chance of developing prostate tumors women who are overweight or

In [ ]:
# Initialize a conversation history tracker
conversation_history = {"disease": None, "previous_questions": []}

def classify_question(question):
    if any(keyword in question.lower() for keyword in ["symptom", "symptoms", "signs"]):
        return "symptoms"
    elif any(keyword in question.lower() for keyword in ["risk", "risks", "factors"]):
        return "risks"
    elif any(keyword in question.lower() for keyword in ["prevent", "prevention", "avoid"]):
        return "prevention"
    else:
        return "general"

def update_context(question):
    # Update disease context if question includes a disease name
    # For simplicity, here we check if the question includes any disease name
    known_diseases = ["diabetes", "hypertension", "asthma"]  # Expand as needed
    for disease in known_diseases:
        if disease in question.lower():
            conversation_history["disease"] = disease
            break

    # Track this question in history
    conversation_history["previous_questions"].append(question)

def generate_answer(question, model, tokenizer, max_length=100):
    # Update context with the new question
    update_context(question)

    # Determine prompt type
    prompt_type = classify_question(question)

    # Use conversation history for context management
    disease = conversation_history["disease"]
    if not disease:
        disease = "the disease"  # Placeholder if no specific disease is identified yet

    # Create custom prompts based on question type and conversation history
    if prompt_type == "general":
        prompt = f"Can you explain {disease} in simple terms?"
    elif prompt_type == "symptoms":
        prompt = f"What are the common symptoms of {disease}?"
    elif prompt_type == "risks":
        prompt = f"Who is at risk for developing {disease}?"
    elif prompt_type == "prevention":
        prompt = f"How can someone prevent {disease}?"
    else:
        prompt = f"Can you explain {disease}?"

    # If it's a follow-up, modify the prompt accordingly
    if len(conversation_history["previous_questions"]) > 1:
        prompt = f"Based on our previous discussion about {disease}, {prompt.lower()}"

    # Prepare input with the dynamic prompt
    input_text = "question: " + prompt
    inputs = tokenizer(input_text, return_tensors="pt").to(device)

    # Generate the answer
    outputs = model.generate(
        inputs['input_ids'],
        max_length=max_length,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        top_k=50,
        top_p=0.95,
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

    # Decode the generated answer
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

# Example usage
question = "What are the symptoms of diabetes?"
print("Answer:", generate_answer(question, model, tokenizer))
question2 = "And the risks?"
print("Answer:", generate_answer(question2, model, tokenizer))


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Answer: diabetes is a group of disorders that affect the bodys ability to produce and use insulin if the condition is not treated or treated the symptoms of diabetes usually begin in childhood or ages 40 and older people usually have no symptoms when symptoms are present they may be symptomatic or have recurrent infections such as swollen lymph nodes or even itchiness numbness or fainting in the hands or feet people with diabetes are at increased risk for developing t
Answer: a woman is at higher risk of developing diabetes than her other than being overweight or obese because of diabetes studies show that if her partner has diabetes in her baby the size of her fetus is normal and will vary widely over time women may have recurrent sex with other people and are at greater risk for developing gi diabetes diabetes is inherited in an autosomal dominant manner in hawaii this is called autonomic diabetes which is
